In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
# from data.py
def get_data (csv_file):
    df = pd.read_csv('../raw_data/' + f'{csv_file}')
    return df

  # OLD VERSION

# NEW VERSION

In [154]:
#from matrix.py ######################### NEW VERSION ###########################################

def encode_categories_articles(clusters_articles_df):  #df doit avoir les champs format ['article_id','catID']
    # création de dummies pour chaque catégorie de produits
    cluster_name=clusters_articles_df.columns[1]
    ohe = OneHotEncoder(sparse=False)# Instanciate encoder
    cat_encoded=pd.DataFrame(ohe.fit_transform(clusters_articles_df[[cluster_name]]))
    cat_names=ohe.categories_
    return pd.concat([articles_hetm[['article_id']],cat_encoded], axis=1) 


def build_segment_base(segment): # ce df contient toutes les transactions des individus du segment 
    return segment[['customer_id']].merge(transact_history)

def build_segment_horizon_transactions(segment_base,X_week,horizon):
    #filtrage sur l'horizon temporel pour connaitre les préférences des customers
    week_min= X_week-horizon+1
    week_max= X_week
    filtre=np.logical_and(segment_base['t_dat'] >= week_min,segment_base['t_dat'] <= week_max)
    Xh_week_achats=segment_base[filtre][['customer_id','t_dat','article_id']]
    Xh_week_achats['counter']=1
    return Xh_week_achats

def build_matrix(Xh_week_achats):
    achats_for_matrix=Xh_week_achats.drop(columns='t_dat')
    #achats_for_matrix['counter']=1 je pense pouvoir supprimer
    achats_for_matrix=achats_for_matrix.merge(articles_categories)

    # je fais un groupby par clients, sans le sort car j'ai ensuite besoin de refaire un tri
    # donc gain de temps de calcul car on suppose que sort_values est plus rapide que le sort de groupby
    achats_gb_cust_cat=achats_for_matrix.groupby(['customer_id'],as_index=False,sort=False)
    achats_gb_cust_cat_srt=achats_gb_cust_cat.sum().sort_values(by=['customer_id','counter'],ascending=False)

    # je refais ensuite un groupby par client pour calculer la somme de chaque dummy de categories
    segment_matrix=achats_gb_cust_cat_srt.drop(columns=['article_id']).groupby(['customer_id'],as_index=False).sum()
    return segment_matrix

def get_max_cats(Xh_week_achats,clusters_articles_df):
    # contrairement à la construction de la matrice, ici je veut les infos de transactions en lignes
    # diversity sera le nbre de catégories utilisées pour proposer 12 références
    tmp=Xh_week_achats.drop(columns='t_dat').merge(clusters_articles_df)
    cluster_name=clusters_articles_df.columns[1]
    achats_for_max_cats=tmp.groupby(['customer_id',cluster_name],as_index=False).sum()
    achats_for_max_cats.sort_values(by=['customer_id','counter'],ascending=False,inplace=True)
    achats_for_max_cats['rank']=achats_for_max_cats.groupby('customer_id')['counter'].rank(method='first',ascending=False)
    return achats_for_max_cats

def top12_per_cat(segment_base,X_week,p,clusters_articles_df):
    # p est le nombre de périodes utilisées pour déterminer le top12 (différent de horizon)
    week_min= X_week-p+1
    week_max= X_week
    filtre=np.logical_and(segment_base['t_dat'] >= week_min,segment_base['t_dat'] <= week_max)
    top12_cat=segment_base[filtre].drop(columns=['customer_id','t_dat']).merge(clusters_articles_df)
    cluster_name=clusters_articles_df.columns[1]
    top12_cat['counter']=1
    top12_cat=top12_cat.groupby([cluster_name,'article_id'],as_index=False).sum()
    top12_cat.sort_values(by=[cluster_name,'counter'],ascending=False,inplace=True)
    top12_cat['rank']=top12_cat.groupby(cluster_name)['counter'].rank(method='first',ascending=False)
    keep_first_12=top12_cat['rank']<=12
    return top12_cat[keep_first_12]

def top12_split(top12_cat_12,diversity): # dans cette version l'algo gère diversity in [1,2,3,4,6,12]
        if diversity==1:
            return top12_cat_12
        else:
            top_nb=12/diversity
            return top12_cat_12[top12_cat_12['rank']<=top_nb]

def predict(top12_cat,diversity):
    # dans cette version l'algo gère diversity in [1,2,3,4,6,12]
    pass


In [296]:
#from baseline0.py
#Baseline_0(S1,article_history,103,104,1)
def Baseline_0(segment,transact_history,X_week,y_week,horizon): # exemple: week104name ='in104' dans l'historique des transactions

    X_week_dummy="in" + f'{X_week}'
    y_week_dummy="in" + f'{y_week}'

    filtre_Xweek=transact_history[X_week_dummy]==1
    #filtre_yweek=transact_history[y_week_dummy]==1

    X_achats=segment[['customer_id']].merge(transact_history[filtre_Xweek][['customer_id','article_id',X_week_dummy]])
    # pour précision : y_achats=segment[['customer_id']].merge(transact_history[filtre_yweek][['customer_id','article_id',y_week_dummy]])

    # calcul des 12 plus populaires du segment:
    articles= X_achats.groupby(['article_id']).sum()
    Top12_arr=articles.sort_values(by=X_week_dummy, ascending=False).head(12)
    Top12=""
    for i in range (0, 12):
        if i==0 :
            Top12=str(Top12_arr.index[i])
        else:
            Top12=Top12 + " "+ str(Top12_arr.index[i])

    #je donne à tous mes customers de mon segment le baseline top12 calculé ci-dessus
    segment['Baseline0']=Top12

    #segment_BS=segment.merge(Baseline0_precision, how = 'left', on='customer_id')

    return segment


In [5]:
# Récupértaion de segments clients de Naceur selon l'âge
# note du 06042022: à ce stade de l'analyse les customers sans information sur l'âge ne sont pas inclus
#get_data('customers_age_16_17.csv')
#get_data('customers_age_18_19.csv','S2')
#get_data('customers_age_20_21.csv','S3')
#get_data('customers_age_22_23.csv','S4')
#get_data('customers_age_24_25.csv','S5')
#get_data('customers_age_26_27.csv','S6')
#get_data('customers_age_28_29.csv','S7')
#get_data('customers_age_30_34.csv','S8')
#get_data('customers_age_35_39.csv','S9')
#get_data('customers_age_40_44.csv','S10')
#set_data('customers_age_45_49.csv','S11')
#get_data('customers_age_50_54.csv','S12')
#get_data('customers_age_55_59.csv','S13')
#get_data('customers_age_60.csv','S14')
S1=get_data('customers_age_16_17.csv')

In [6]:
# Récupération du petit fichier de transactions "test" contenant semaines 101 à 104:
transactions_file='transaction_train_week_101_104.csv'
transactions_file_path=f'../raw_data/{transactions_file}'
transact_history=get_data(transactions_file_path)

In [7]:
#cleaning du petit fichier utilisé pour tester les programmes
transactions_101_104 = transact_history
transactions_101_104.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'price', 'sales_channel_id'], inplace=True)

In [8]:
#construction de la base_segment utile pour le calcul des valeurs des éléments de la matrice
S1_base=build_segment_base(S1)

,customer_id,t_dat,article_id
0,0006ca2c5ecdbdcc1de5e72adffc6c33448fc2aaa62afe...,101,572998001
1,0006ca2c5ecdbdcc1de5e72adffc6c33448fc2aaa62afe...,101,572998009
2,00089f13f465ec902e5c49a3bb408c5e31205096d6f267...,101,926502001
3,000e3f587242eb077685a487ad27dad632a4801576dfd1...,101,501620049
4,001c42459c8978d55a69fa1528975893b011a6539631f3...,101,885077003


In [31]:
S1_base

,customer_id,t_dat,article_id
0,0006ca2c5ecdbdcc1de5e72adffc6c33448fc2aaa62afe...,101,572998001
1,0006ca2c5ecdbdcc1de5e72adffc6c33448fc2aaa62afe...,101,572998009
2,00089f13f465ec902e5c49a3bb408c5e31205096d6f267...,101,926502001
3,000e3f587242eb077685a487ad27dad632a4801576dfd1...,101,501620049
4,001c42459c8978d55a69fa1528975893b011a6539631f3...,101,885077003
...,...,...,...
5950,ffd6bb6d6a33e415467e69d18d28ec9e64061a4f58d67c...,104,914441006
5951,ffd6bb6d6a33e415467e69d18d28ec9e64061a4f58d67c...,104,914441004
5952,ffec95723bd3cfc50cef76f198716d18e395261f7cd8b1...,104,858147006
5953,ffec95723bd3cfc50cef76f198716d18e395261f7cd8b1...,104,892643001


In [9]:
#création des clusters d'articles
articles_hetm = get_data('../raw_data/articles.csv')
articles_hetm=articles_hetm[['article_id','garment_group_name']] #fait office de clusters_articles_df
articles_categories=encode_categories_articles(articles_hetm)

In [10]:
articles_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   article_id  105542 non-null  int64  
 1   0           105542 non-null  float64
 2   1           105542 non-null  float64
 3   2           105542 non-null  float64
 4   3           105542 non-null  float64
 5   4           105542 non-null  float64
 6   5           105542 non-null  float64
 7   6           105542 non-null  float64
 8   7           105542 non-null  float64
 9   8           105542 non-null  float64
 10  9           105542 non-null  float64
 11  10          105542 non-null  float64
 12  11          105542 non-null  float64
 13  12          105542 non-null  float64
 14  13          105542 non-null  float64
 15  14          105542 non-null  float64
 16  15          105542 non-null  float64
 17  16          105542 non-null  float64
 18  17          105542 non-null  float64
 19  18

In [11]:
articles_categories.head()

,article_id,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
0,108775015,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108775044,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,108775051,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,110065001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,110065002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [74]:
S1_Xh_week_achats=build_segment_horizon_transactions(S1_base,103,1)

In [75]:
S1_Xh_week_achats.shape

(885, 4)

In [76]:
S1_Xh_week_achats

,customer_id,t_dat,article_id,counter
18,00953d607bb9b498cf020c314436b74c54876e1f03739a...,103,806388003,1
19,00953d607bb9b498cf020c314436b74c54876e1f03739a...,103,806388018,1
20,00953d607bb9b498cf020c314436b74c54876e1f03739a...,103,806388001,1
21,00953d607bb9b498cf020c314436b74c54876e1f03739a...,103,806388001,1
22,00953d607bb9b498cf020c314436b74c54876e1f03739a...,103,866731001,1
...,...,...,...,...
5866,fd458cc672b5dbd88d7cd1e92bb289f1a5f20d12bcda61...,103,853881001,1
5871,fddef481e1e792752bb5b7243c1345cb0379a74c64722d...,103,621381014,1
5899,fe452b72e97790e68dcf79f49521782d486c345981f773...,103,487800051,1
5907,feb01a2d17ca601bb4361099013b9010773778a3da3552...,103,706016003,1


In [78]:
build_matrix(S1_Xh_week_achats)

,customer_id,counter,0,1,2,3,4,5,6,7,...,11,12,13,14,15,16,17,18,19,20
0,00953d607bb9b498cf020c314436b74c54876e1f03739a...,5,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0105f638a3d98cd1918028de84d296e2ec5f075d5ccf3b...,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,017e589a1b5990b94288741a86252e14c1c361f3802827...,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,026e36e2a2131e0bdeb78752d03c3caad8932f0e361a3a...,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18...,4,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,fd458cc672b5dbd88d7cd1e92bb289f1a5f20d12bcda61...,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
340,fddef481e1e792752bb5b7243c1345cb0379a74c64722d...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
341,fe452b72e97790e68dcf79f49521782d486c345981f773...,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
342,feb01a2d17ca601bb4361099013b9010773778a3da3552...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [142]:
achats_for_max_cats=get_max_cats(S1_Xh_week_achats,articles_hetm)

In [143]:
achats_for_max_cats

,customer_id,garment_group_name,article_id,counter,rank
601,fede72be6d8a2130624098f344c15f4d88a45d8b3cbbc9...,Accessories,896123001,1,1.0
600,feb01a2d17ca601bb4361099013b9010773778a3da3552...,Trousers,706016003,1,1.0
599,fe452b72e97790e68dcf79f49521782d486c345981f773...,Jersey Basic,487800051,1,1.0
598,fddef481e1e792752bb5b7243c1345cb0379a74c64722d...,Trousers,621381014,1,1.0
596,fd458cc672b5dbd88d7cd1e92bb289f1a5f20d12bcda61...,Jersey Fancy,853881001,1,1.0
...,...,...,...,...,...
4,026e36e2a2131e0bdeb78752d03c3caad8932f0e361a3a...,Dresses Ladies,832084003,1,1.0
3,017e589a1b5990b94288741a86252e14c1c361f3802827...,Accessories,639448001,1,1.0
2,0105f638a3d98cd1918028de84d296e2ec5f075d5ccf3b...,Jersey Basic,685813043,1,1.0
0,00953d607bb9b498cf020c314436b74c54876e1f03739a...,Jersey Basic,3225552023,4,1.0


In [144]:
achats_for_max_cats[achats_for_max_cats['customer_id']=='0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18f0ca611d4938ab1a48']

,customer_id,garment_group_name,article_id,counter,rank
5,0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18...,Jersey Basic,1431248020,2,1.0
6,0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18...,Trousers,1791672006,2,2.0


In [157]:
top12_cat_12=top12_per_cat(S1_base,103,1,articles_hetm)

In [158]:
top12_cat_12

,garment_group_name,article_id,counter,rank
594,Unknown,794321007,1,1.0
595,Unknown,815580001,1,2.0
596,Unknown,874937001,1,3.0
557,"Under-, Nightwear",748355003,4,1.0
558,"Under-, Nightwear",748355018,3,2.0
...,...,...,...,...
42,Accessories,910264001,2,8.0
0,Accessories,337991001,1,9.0
1,Accessories,516859013,1,10.0
2,Accessories,524465001,1,11.0


In [159]:
# vérification du rang sur certaines catégories
top12_cat_12['garment_group_name']=='Trousers'
top12_cat_12[top12_cat_12['garment_group_name']=='Trousers']

,garment_group_name,article_id,counter,rank
447,Trousers,621381012,12,1.0
448,Trousers,621381014,6,2.0
457,Trousers,706016002,6,3.0
433,Trousers,448509014,5,4.0
456,Trousers,706016001,5,5.0
438,Trousers,562245046,4,6.0
470,Trousers,760084003,4,7.0
487,Trousers,895836003,4,8.0
432,Trousers,448509001,3,9.0
441,Trousers,572998013,3,10.0


In [160]:
top12_cat_12['garment_group_name']=='Trousers Denim'
top12_cat_12[top12_cat_12['garment_group_name']=='Trousers Denim']

,garment_group_name,article_id,counter,rank
510,Trousers Denim,636323001,5,1.0
526,Trousers Denim,864216001,4,2.0
516,Trousers Denim,729860002,3,3.0
540,Trousers Denim,899749005,3,4.0
515,Trousers Denim,724906019,2,5.0
520,Trousers Denim,797835008,2,6.0
528,Trousers Denim,864224005,2,7.0
529,Trousers Denim,867966002,2,8.0
532,Trousers Denim,875272012,2,9.0
535,Trousers Denim,882899003,2,10.0


In [163]:
top12=top12_split(top12_cat_12,1)
top12

,garment_group_name,article_id,counter,rank
594,Unknown,794321007,1,1.0
595,Unknown,815580001,1,2.0
596,Unknown,874937001,1,3.0
557,"Under-, Nightwear",748355003,4,1.0
558,"Under-, Nightwear",748355018,3,2.0
...,...,...,...,...
42,Accessories,910264001,2,8.0
0,Accessories,337991001,1,9.0
1,Accessories,516859013,1,10.0
2,Accessories,524465001,1,11.0


In [164]:
top6=top12_split(top12_cat_12,2)
top6

,garment_group_name,article_id,counter,rank
594,Unknown,794321007,1,1.0
595,Unknown,815580001,1,2.0
596,Unknown,874937001,1,3.0
557,"Under-, Nightwear",748355003,4,1.0
558,"Under-, Nightwear",748355018,3,2.0
...,...,...,...,...
21,Accessories,759482001,3,2.0
40,Accessories,907409001,3,3.0
46,Accessories,928839001,3,4.0
4,Accessories,552716012,2,5.0


In [165]:
top6['garment_group_name']=='Trousers Denim'
top6[top6['garment_group_name']=='Trousers Denim']

,garment_group_name,article_id,counter,rank
510,Trousers Denim,636323001,5,1.0
526,Trousers Denim,864216001,4,2.0
516,Trousers Denim,729860002,3,3.0
540,Trousers Denim,899749005,3,4.0
515,Trousers Denim,724906019,2,5.0
520,Trousers Denim,797835008,2,6.0


In [166]:
top4=top12_split(top12_cat_12,3)
top4

,garment_group_name,article_id,counter,rank
594,Unknown,794321007,1,1.0
595,Unknown,815580001,1,2.0
596,Unknown,874937001,1,3.0
557,"Under-, Nightwear",748355003,4,1.0
558,"Under-, Nightwear",748355018,3,2.0
...,...,...,...,...
52,Blouses,697564006,1,4.0
9,Accessories,639448001,3,1.0
21,Accessories,759482001,3,2.0
40,Accessories,907409001,3,3.0


In [167]:
top3=top12_split(top12_cat_12,4)
top3

,garment_group_name,article_id,counter,rank
594,Unknown,794321007,1,1.0
595,Unknown,815580001,1,2.0
596,Unknown,874937001,1,3.0
557,"Under-, Nightwear",748355003,4,1.0
558,"Under-, Nightwear",748355018,3,2.0
590,"Under-, Nightwear",923037001,3,3.0
510,Trousers Denim,636323001,5,1.0
526,Trousers Denim,864216001,4,2.0
516,Trousers Denim,729860002,3,3.0
447,Trousers,621381012,12,1.0


In [168]:
top2=top12_split(top12_cat_12,6)
top2

,garment_group_name,article_id,counter,rank
594,Unknown,794321007,1,1.0
595,Unknown,815580001,1,2.0
557,"Under-, Nightwear",748355003,4,1.0
558,"Under-, Nightwear",748355018,3,2.0
510,Trousers Denim,636323001,5,1.0
526,Trousers Denim,864216001,4,2.0
447,Trousers,621381012,12,1.0
448,Trousers,621381014,6,2.0
427,Swimwear,559633008,1,1.0
428,Swimwear,685605001,1,2.0


In [169]:
top1=top12_split(top12_cat_12,12)
top1

,garment_group_name,article_id,counter,rank
594,Unknown,794321007,1,1.0
557,"Under-, Nightwear",748355003,4,1.0
510,Trousers Denim,636323001,5,1.0
447,Trousers,621381012,12,1.0
427,Swimwear,559633008,1,1.0
425,Special Offers,644288001,2,1.0
423,Socks and Tights,889703001,3,1.0
405,Skirts,869379007,2,1.0
392,Shoes,619234005,1,1.0
389,Shirts,743840018,1,1.0
